In [1]:
# check NVCC version
!nvcc -V

# check GCC version
!gcc --version

# check python in conda environment
!which python

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Thu_Feb_10_18:23:41_PST_2022
Cuda compilation tools, release 11.6, V11.6.112
Build cuda_11.6.r11.6/compiler.30978841_0
gcc (GCC) 11.2.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/home/abs/anaconda3/envs/open-mmlab3/bin/python


In [2]:
!pwd
%cd ..
!pwd

/home/abs/mmpose/demo
/home/abs/mmpose
/home/abs/mmpose


In [3]:
# Check Pytorch installation
import torch, torchvision
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)


# Check MMPose installation
import mmpose
print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 1.8.2 True
torchvision version: 0.9.2
mmpose version: 0.23.0


ImportError: /home/abs/anaconda3/envs/open-mmlab3/lib/python3.8/site-packages/mmcv/_ext.cpython-38-x86_64-linux-gnu.so: undefined symbol: _ZNSt15__exception_ptr13exception_ptr9_M_addrefEv

In [4]:
import numpy as np
import cv2
import joblib
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
local_runtime = False

try:
  from google.colab.patches import cv2_imshow  # for image visualization in colab
except:
  local_runtime = True

seg_config_file = '../mmsegmentation/configs/deeplabv3plus/deeplabv3plus_r101-d8_512x512_40k_voc12aug.py'
seg_checkpoint_file = '../mmsegmentation/work_dirs/deeplabv3plus_r101-d8_512x512_40k_voc12aug/iter_40000.pth'

rear_pose_config = 'configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/animalpose/res152_animalpose_256x256.py'
rear_pose_checkpoint = '/home/abs/mmpose/work_dirs_rear/res152_animalpose_256x256/epoch_210.pth'
side_pose_config = '/home/abs/mmposeSide/configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/animalpose/res152_animalpose_256x256.py'
side_pose_checkpoint = '/home/abs/mmposeSide/work_dirs/res152_animalpose_256x256/epoch_210.pth'
det_config = 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
weight_filename = "/home/abs/mmsegmentation/work_dirs/weight_model/alpha_model_april_2022.joblib"
# initialize seg & pose model
# build the model from a config file and a checkpoint file
model = init_segmentor(seg_config_file, seg_checkpoint_file, device='cuda:0')

rear_pose_model = init_pose_model(rear_pose_config, rear_pose_checkpoint)
side_pose_model = init_pose_model(side_pose_config, side_pose_checkpoint)

# initialize detector
rear_det_model = init_detector(det_config, det_checkpoint)
side_det_model = init_detector(det_config, det_checkpoint)


rear_img = 'tests/data/cow/6.0_r_96_5.0_M.jpg'
side_img = 'tests/data/cow/6.0_s_96_5.0_M.jpg'

# inference detection
rear_mmdet_results = inference_detector(rear_det_model, rear_img)
side_mmdet_results = inference_detector(side_det_model, side_img)
print(rear_mmdet_results)
# extract person (COCO_ID=1) bounding boxes from the detection results
rear_person_results = process_mmdet_results(rear_mmdet_results, cat_id=20)
side_person_results = process_mmdet_results(side_mmdet_results, cat_id=20)

# inference Segmentation
seg_result = inference_segmentor(model, side_img)


seg = np.asarray(seg_result)
sticker = cattle = bg = 0


sticker = (seg == 0).sum()

cattle = (seg == 1).sum()

bg =( seg == 2).sum()


print(sticker)
print(cattle)
print(bg)

# inference pose
rear_pose_results, rear_returned_outputs = inference_top_down_pose_model(rear_pose_model,
                                                               rear_img,
                                                               rear_person_results,
                                                               bbox_thr=0.3,
                                                               format='xyxy',
                                                               dataset=rear_pose_model.cfg.data.test.type)
side_pose_results, side_returned_outputs = inference_top_down_pose_model(side_pose_model,
                                                               side_img,
                                                               side_person_results,
                                                               bbox_thr=0.3,
                                                               format='xyxy',
                                                               dataset=side_pose_model.cfg.data.test.type)

# KPT rear and side
rear_kpt = rear_pose_results[0]["keypoints"][:,0:2]
side_kpt = side_pose_results[0]["keypoints"][:,0:2]

rearKptID=rearx0=reary0=rearx1=reary1=rearx2=reary2=rearx3=reary3=0
sideKptID=sidex0=sidey0=sidex1=sidey1=sidex5=sidey5=sidex2=sidey2=sidex4=sidey4=sidex3=sidey3=0

for kptx,kpty in rear_kpt:
  if rearKptID == 0:
      rearx0 = kptx
      reary0 = kpty
  elif rearKptID == 1:
      rearx1 = kptx
      reary1 = kpty
  elif rearKptID == 2:
      rearx2 = kptx
      reary2 = kpty
  elif rearKptID == 3:
      rearx3 = kptx
      reary3 = kpty
  
  rearKptID+=1

for kptx,kpty in side_kpt:

  if sideKptID == 0:
      sidex0 = kptx
      sidey0 = kpty
  elif sideKptID == 1:
      sidex1 = kptx
      sidey1 = kpty
  elif sideKptID == 2:
      sidex5 = kptx
      sidey5 = kpty
  elif sideKptID == 3:
      sidex2 = kptx
      sidey2 = kpty
  elif sideKptID == 4:
      sidex4 = kptx
      sidey4 = kpty
  elif sideKptID == 5:
      sidex3 = kptx
      sidey3 = kpty   

  sideKptID+=1

side_Length = round(((sidey1-sidey0)**2+(sidex1-sidex0)**2)**0.5)
side_F_Girth = round(((sidey3-sidey2)**2+(sidex3-sidex2)**2)**0.5)
side_R_Girth = round(((sidey5-sidey4)**2+(sidex5-sidex4)**2)**0.5)
rear_width = round(((reary1-reary0)**2+(rearx1-rearx0)**2)**0.5)
rear_height = round(((reary3-reary2)**2+(rearx3-rearx2)**2)**0.5)


print(f'side_Length = {side_Length}, side_F_Girth = {side_F_Girth}, side_R_Girth = {side_R_Girth}, rear_Width = {rear_width}, rear_Height = {rear_height}')


# Weight prediction

loaded_model = joblib.load(weight_filename)
print(loaded_model.predict([[side_Length,	side_F_Girth,	side_R_Girth,	rear_width,	sticker,	cattle]]))



# show pose estimation results
rear_vis_result = vis_pose_result(rear_pose_model,
                             rear_img,
                             rear_pose_results,
                             thickness=1,
                             dataset=rear_pose_model.cfg.data.test.type,
                             show=False)
side_vis_result = vis_pose_result(side_pose_model,
                             side_img,
                             side_pose_results,
                             thickness=1,
                             dataset=side_pose_model.cfg.data.test.type,
                             show=False)
# reduce image size
rear_vis_result = cv2.resize(rear_vis_result, dsize=None, fx=0.5, fy=0.5)
side_vis_result = cv2.resize(side_vis_result, dsize=None, fx=0.5, fy=0.5)


if local_runtime:
  from IPython.display import Image, display
  import tempfile
  import os.path as osp
  with tempfile.TemporaryDirectory() as tmpdir:
    rear_file_name = osp.join(tmpdir, 'rear_pose_results.png')
    side_file_name = osp.join(tmpdir, 'side_pose_results.png')

    cv2.imwrite(rear_file_name, rear_vis_result)
    cv2.imwrite(side_file_name, side_vis_result)

    display(Image(rear_file_name))
    display(Image(side_file_name))
    show_result_pyplot(model, side_img, seg_result, get_palette('voc'))


else:
  show_result_pyplot(model, side_img, seg_result, get_palette('voc'))
  cv2_imshow(rear_vis_result)
  cv2_imshow(side_vis_result)





ImportError: /home/abs/anaconda3/envs/open-mmlab3/lib/python3.8/site-packages/mmcv/_ext.cpython-38-x86_64-linux-gnu.so: undefined symbol: _ZNSt15__exception_ptr13exception_ptr9_M_addrefEv

In [ ]:
import cv2
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
local_runtime = False

try:
  from google.colab.patches import cv2_imshow  # for image visualization in colab
except:
  local_runtime = True

# side_pose_config = '/home/abs/mmposeSide/configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/animalpose/res152_animalpose_256x256.py'
# side_pose_checkpoint = '/home/abs/mmposeSide/work_dirs/res152_animalpose_256x256/epoch_210.pth'
side_pose_config = 'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
side_pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'

det_config = 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

# initialize pose model

side_pose_model = init_pose_model(side_pose_config, side_pose_checkpoint)

# initialize detector

side_det_model = init_detector(det_config, det_checkpoint)



side_img = '/home/abs/mmpose/data/Side/images/6.0_s_96_5.0_M.jpg'

# inference detection
side_mmdet_results = inference_detector(side_det_model, side_img)

# extract person (COCO_ID=1) bounding boxes from the detection results
side_person_results = process_mmdet_results(side_mmdet_results, cat_id=20)

# inference pose
side_pose_results, side_returned_outputs = inference_top_down_pose_model(side_pose_model,
                                                               side_img,
                                                               side_person_results,
                                                               bbox_thr=0.3,
                                                               format='xyxy',
                                                               dataset=side_pose_model.cfg.data.test.type)

print(side_pose_results[0]["keypoints"])
# show pose estimation results

side_vis_result = vis_pose_result(side_pose_model,
                             side_img,
                             side_pose_results,
                             thickness=1,
                             dataset=side_pose_model.cfg.data.test.type,
                             show=False)
# reduce image size
side_vis_result = cv2.resize(side_vis_result, dsize=None, fx=0.5, fy=0.5)


if local_runtime:
  from IPython.display import Image, display
  import tempfile
  import os.path as osp
  with tempfile.TemporaryDirectory() as tmpdir:
    side_file_name = osp.join(tmpdir, 'side_pose_results.png')
    cv2.imwrite(side_file_name, side_vis_result)

    display(Image(side_file_name))

else:
  cv2_imshow(side_vis_result)





In [5]:
%cd work_dirs/res152_animalpose_256x256/

/home/abs/mmpose/work_dirs/res152_animalpose_256x256


In [6]:
ls

20220303_155519.log       20220303_170840.log.json  20220303_183120.log
20220303_155749.log       20220303_171008.log       20220306_104116.log
20220303_160001.log       20220303_171008.log.json  20220306_104309.log
20220303_160356.log       20220303_171452.log       20220306_104902.log
20220303_160445.log       20220303_171452.log.json  20220306_105003.log
20220303_162730.log       20220303_171813.log       20220306_105037.log
20220303_163306.log       20220303_171813.log.json  20220306_105110.log
20220303_163352.log       20220303_181437.log       20220306_105129.log
20220303_163604.log       20220303_181701.log       20220306_105632.log
20220303_164552.log       20220303_181835.log       20220306_110051.log
20220303_164637.log       20220303_182000.log       20220306_110235.log
20220303_164719.log       20220303_182217.log       20220306_110235.log.json
20220303_165832.log       20220303_182226.log       best_AP_epoch_50.pth
20220303_170432.log       20220303_182822.log       epoch_